# New dynamics

## Loading modules

In [ ]:
using QuantumWalk
using LightGraphs
using SparseArrays
using LinearAlgebra

## QWPeriod

The main purpose of the package is to provide easy extendability of the code. As
Quantum walk models needs to be adjusted to existing dynamics, the key is to defined
an describe dynamic. As an example we propose an example of simple dynamic determining
the period of the model.

We start by introducing a general type representing the dynamic. 

In [ ]:
struct QWPeriod{T} <: QWDynamics{T}
  model::T
  parameters::Dict{Symbol,Any}

  function QWPeriod(model::T,
                    parameters::Dict{Symbol}) where T<:QWModelDiscr
    check_qwdynamics(QWPeriod, model, parameters)
    new{T}(model, parameters)
  end
end

We can see, that we need to implement `check_qwdynamics` function. Yet the function inherits the `execute` family functions from `QWDynamics` type. Note the type accepts only discrete quantum walk evolution - while the problem can be stated for the continuous-time as well, its implementation may be much more complicated.

Let us define the basic function determining the period.

In [ ]:
function determine_period(qwp::QWPeriod,
                          init_state,
                          state_diff_val::Real,
                          tmax::Int=nv(graph(qwp)))
  @assert tmax>0 "maximal time needs to be greater than 0"
  state = evolve(qwp, init_state)
  for t=1:tmax
    if state_diff(qwp, state, init_state) < state_diff_val
      return t
    end
    state = evolve(qwp, state)
  end
  return -1
end

The function above single execute the evolution as long as the upper limit for time is achieved, or the state is close to the initial state. according to the measure for `state_diff`.

According to the definition above, the following function should be implemented for `Model`:
* `QWPeriod(model::Model,...)`,
* `check_dynamics(QWPeriod, model::Model, parameters::Dict{Symbol,Any})`,
* `evolve(qwp::QWPeriod{<:Model}, state)`,
* `state_diff(qwp::QWPeriod{<:Model}, state1::S, state2::S) where S`.
Note that for `Szegedy` function `evolve` is already implemented for `QWDynamics`, and we can use
private function for `check_dynamics` already implemented. 

The implementation of missing function for the Szegedy can have following form:

In [ ]:
function QWPeriod(szegedy::AbstractSzegedy)
  operators = QuantumWalk.szegedy_walk_operators(szegedy)
  parameters = Dict{Symbol,Any}(:operators => operators)
  QWPeriod(szegedy, parameters)
end

function check_qwdynamics(::Type{QWPeriod}, szegedy::AbstractSzegedy, parameters::Dict{Symbol})
  QuantumWalk.check_szegedy(szegedy, parameters)
end

function state_diff(qwp::QWPeriod{<:AbstractSzegedy},
                    state1::SparseVector{T},
                    state2::SparseVector{T}) where T<:Number
  1-abs(sum(state1 .* conj.(state2)))
end

Thanks to the code above we can check the periodicity for the `Szegedy` walk.

In [ ]:
n = 20
qwp = QWPeriod(Szegedy(barabasi_albert(n, 3)))
determine_period(qwp, sparse(fill(1/n, n^2)), 0.01)
state = sparse(randn(n^2))
state /= norm(state)
determine_period(qwp, state, 0.05, 8000)

## Generalizing the periodic determination implementation

Note that the periodic computation can be generalized for arbitrary quantum walk dynamics - we do not need to implement special type.
            Hence we can actually implmement period determination in following form.
            

In [ ]:
function determine_period(qwp::QWDynamics{<:QWModelDiscr},
                          init_state,
                          state_diff_val::Real,
                          tmax::Int=nv(graph(qwp)))
  @assert tmax >0 "maximal time needs to be greater than 0"
  state = evolve(qwp, init_state)
  for t=1:tmax
    if state_diff(qwp, state, init_state) < state_diff_val
      return t
    end
    state = evolve(qwp, state)
  end
  return -1
end

function state_diff(qwp::QWDynamics{<:AbstractSzegedy},
                    state1::AbstractArray{T},
                    state2::AbstractArray{T}) where T<:Number
  1-abs(sum(state1 .* conj.(state2)))
end

Thank to the function above we can calculate the periodicity of `QuantumSearch`.

In [ ]:
n = 20
qwp = QWSearch(Szegedy(barabasi_albert(n, 3)), [1])
println("Uniform state")
println(determine_period(qwp, sparse(fill(1/n, n^2)), 0.01))

state = sparse(randn(n^2))
state /= norm(state)
println("Random state")
println(determine_period(qwp, state, 0.05, 8000))

Furthermore, since for quantum spatial search there exists a special inital state, we can create additional function
as follows.


In [ ]:
function determine_period(qwp::QWSearch{<:QWModelDiscr},
                          state_diff_val::Real,
                          tmax::Int=nv(graph(qwp)))
  determine_period(qwp, sparse(initial_state(qwp)), 0.05, 8000)
end


Which results in small usage improvement.

In [ ]:
qwp = QWSearch(Szegedy(CycleGraph(30)), [1])
determine_period(qwp, 0.01)